In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3

import sys
sys.path.append('../src/')
sys.path.append('/notebook')

from models import combined_cnn
from loss import smooth_labels
from metrics import sensitivity, specificity

/Users/anthonyrentsch/anaconda3/envs/somerville-parking/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/anthonyrentsch/anaconda3/envs/somerville-parking/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/anthonyrentsch/anaconda3/envs/somerville-parking/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

In [2]:
# import dataframe with filenames and labels
sample = pd.read_csv('../labels/training_labels_updated_111219.csv')

In [3]:
# smooth labels
sample['full_label'] = 'aerial_' + sample['AERIAL_Driveway'].astype(int).astype(str) + \
                       '_gsv_' + sample['GSV_Driveway'].astype(int).astype(str)
sample['smooth_label'] = smooth_labels(sample['full_label'])

In [4]:
# apply 
sample['temp_label'] = sample['final_label'].apply(lambda x: np.round(x))
sample['temp_label'] = sample['temp_label'].astype('int').astype('str')
sample['final_label'] =  sample['final_label'].astype('str')

In [5]:
# peak at data
sample.head()

,ADDR_NUM,FULL_STR,MBL,ADDRESS_FOR_STREET_VIEW,AERIAL_Driveway,GSV_Driveway,final_label,aerial_filename,gsv_filename,full_label,smooth_label,temp_label
0,5,DOUGLAS AVE,47-G-17,"5 DOUGLAS AVE, SOMERVILLE, MA",2.0,0.0,0.1,5_DOUGLAS_AVE_aerial.png,5_DOUGLAS_AVE.jpg,aerial_2_gsv_0,0.1,0
1,6,HILLSIDE CIR,38-A-47,"6 HILLSIDE CIR, SOMERVILLE, MA",1.0,2.0,0.9,6_HILLSIDE_CIR_aerial.png,6_HILLSIDE_CIR.jpg,aerial_1_gsv_2,0.9,1
2,8,BELMONT PL,44-A-48,"8 BELMONT PL, SOMERVILLE, MA",2.0,2.0,0.5,8_BELMONT_PL_aerial.png,8_BELMONT_PL.jpg,aerial_2_gsv_2,0.5,0
3,8,DRESDEN CIR,25-A-13,"8 DRESDEN CIR, SOMERVILLE, MA",2.0,2.0,0.5,8_DRESDEN_CIR_aerial.png,8_DRESDEN_CIR.jpg,aerial_2_gsv_2,0.5,0
4,8,HAWTHORNE ST,25-D-16,"8 HAWTHORNE ST, SOMERVILLE, MA",0.0,2.0,0.1,8_HAWTHORNE_ST_aerial.png,8_HAWTHORNE_ST.jpg,aerial_0_gsv_2,0.1,0


In [6]:
# split train and test
train_data, test_data = train_test_split(sample, test_size = 0.25, random_state = 100)

In [21]:
# parameters
aer_image_dim = (128, 128, 3) # (128, 128, 4)
aerial_dir = '../data/training/aerial_images/'
x_column = 'aerial_filename'
y_column = 'smooth_label'
batch_size = 32

In [22]:
# data augmentation and generator
## train
aerial_gen_obj = ImageDataGenerator(horizontal_flip = True, vertical_flip = True, 
                                    width_shift_range = 0.1, height_shift_range = 0.1, 
                                    zoom_range = 0.1, rotation_range = 40)
train_generator = aerial_gen_obj.flow_from_dataframe(train_data, directory = aerial_dir, 
                                                x_col= x_column, y_col= y_column, 
                                                target_size=(aer_image_dim[0], aer_image_dim[1]), 
                                                color_mode='rgba', class_mode='raw', #class_mode='binary'
                                                batch_size=batch_size, 
                                                shuffle=True, seed=100)

## test
test_generator = aerial_gen_obj.flow_from_dataframe(test_data, directory = aerial_dir, 
                                                x_col= x_column, y_col= y_column, 
                                                target_size=(aer_image_dim[0], aer_image_dim[1]), 
                                                color_mode='rgba', class_mode='raw', 
                                                batch_size=batch_size, 
                                                shuffle=True, seed=100)

Found 366 validated image filenames.
Found 122 validated image filenames.


In [53]:
# define models
def satellite_cnn(image_dim = aer_image_dim):
    
    sat_input_img = layers.Input(shape=image_dim, name='aerial_image_input')
    sat_cnn = layers.Conv2D(256, (3,3), activation = 'relu')(sat_input_img)
    sat_cnn = layers.MaxPooling2D((2,2))(sat_cnn)
    sat_cnn = layers.Conv2D(128, (3,3), activation = 'relu')(sat_cnn)
    sat_cnn = layers.MaxPooling2D((2,2))(sat_cnn)
    sat_cnn = layers.Conv2D(64, (3,3), activation = 'relu')(sat_cnn)
    sat_cnn = layers.MaxPooling2D((2,2))(sat_cnn)
    sat_cnn = layers.Conv2D(32, (3,3), activation = 'relu')(sat_cnn)
    sat_cnn = layers.MaxPooling2D((2,2))(sat_cnn)
    sat_cnn = layers.Conv2D(16, (3,3), activation = 'relu')(sat_cnn)
    sat_flat = layers.Flatten()(sat_cnn)
    sat_image_embedding = layers.Dense(200, activation='relu')(sat_flat)
    
    full_embedding = layers.Dense(300, activation='relu')(sat_image_embedding)
    full_embedding = layers.Dense(100, activation='relu')(full_embedding)
    output = layers.Dense(1, activation='sigmoid')(full_embedding)
    
    model = models.Model(inputs=sat_input_img, outputs=output)
    adam = optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', sensitivity, specificity])
    
    return model

def satellite_cnn_transfer(image_dim = aer_image_dim, pre_trained = None):
    
    if not pre_trained:
        print('No model specified')
        return 
        
    sat_input_img = layers.Input(shape=image_dim, name='aerial_image_input')
    print(sat_input_img)
    sat_input_img = layers.Lambda(lambda x: x[:,:,:,0:2], input_shape=image_dim)(sat_input_img)
    print(sat_input_img)
    
    for layer in pre_trained.layers:
        layer.trainable = False
    pre_trained_embedding = pre_trained(sat_input_img)
    pre_trained_embedding = layers.Flatten()(pre_trained_embedding)
    
    full_embedding = layers.Dense(300, activation='relu')(pre_trained_embedding)
    full_embedding = layers.Dense(100, activation='relu')(full_embedding)
    output = layers.Dense(1, activation='sigmoid')(full_embedding)
    
    model = models.Model(inputs=sat_input_img, outputs=output)
    adam = optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', sensitivity, specificity])
    
    return model

In [54]:
# load Inception V3 Image Net weights
# inception_v3 = InceptionV3(include_top=False, input_shape=aer_image_dim)

In [55]:
# instantiate model
# model = satellite_cnn()
model = satellite_cnn_transfer(pre_trained = inception_v3)
model.summary()

Tensor("aerial_image_input_14:0", shape=(?, 128, 128, 3), dtype=float32)
Tensor("lambda_13/strided_slice:0", shape=(?, 128, 128, 2), dtype=float32)


ValueError: Dimensions must be equal, but are 2 and 3 for 'inception_v3_10/conv2d_564/Conv2D' (op: 'Conv2D') with input shapes: [?,128,128,2], [3,3,3,32].

In [15]:
# training parameters
epochs = 10
val_steps = 5

In [25]:
# train model
history = model.fit_generator(generator=train_generator, 
                              validation_data=test_generator, 
                              validation_steps=val_steps, 
                              epochs=epochs,
                              steps_per_epoch=np.ceil(train_data.shape[0]//batch_size),
                              verbose=1)

Epoch 1/10


ValueError: Error when checking input: expected aerial_image_input to have shape (128, 128, 3) but got array with shape (128, 128, 4)

In [ ]:
# save model
# model.save('../models/satellite_model.h5')

In [ ]:
# # load model
# from tensorflow.keras.models import load_model
# model = load_model('../models/satellite_model.h5', 
#                    custom_objects={'sensitivity': sensitivity, 'specificity': specificity})

In [ ]:
train_baseline_acc = train_data['temp_label'].astype('float').mean()
test_baseline_acc = test_data['temp_label'].astype('float').mean()

In [ ]:
# plot performance
fig, ax = plt.subplots(1, 3, figsize=(25,5))

ax[0].plot(history.history['acc'], color='darkblue', label='train')
ax[0].plot(history.history['val_acc'], color='darkred', label='val')
ax[0].axhline(train_baseline_acc, color='darkblue', linestyle='dashed', label='train baseline')
ax[0].axhline(test_baseline_acc, color='darkred', linestyle='dashed', label='val baseline')
ax[0].legend()
ax[0].set_title('Accuracy')

ax[1].plot(history.history['sensitivity'], color='darkblue', label='train')
ax[1].plot(history.history['val_sensitivity'], color='darkred', label='val')
ax[1].legend()
ax[1].set_title('Sensitivity')

ax[2].plot(history.history['specificity'], color='darkblue', label='train')
ax[2].plot(history.history['val_specificity'], color='darkred', label='val')
ax[2].legend()
ax[2].set_title('Specificity');

In [ ]:
# score training set and test set (even though it was used for validation)
train_preds = model.predict_generator(train_generator)
test_preds = model.predict_generator(test_generator)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20,10))

ax.hist(train_preds, color='darkblue', alpha=0.5, label='train scores')
ax.hist(test_preds, color='darkred', alpha=0.5, label='test scores')
ax.legend()
ax.set_xlabel('Score')
ax.set_title('Distribution of model scores');

---